In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from brisc.manuscript_analysis import connectivity_matrices as conn_mat
from brisc.manuscript_analysis import distance_between_cells as dist
from brisc.manuscript_analysis import bootstrapping as boot
from brisc.manuscript_analysis import load
from brisc.exploratory_analysis.plot_summary_for_all_bc import (
    compute_flatmap_coors,
    get_avg_layer_depth,
)

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

matplotlib.rcParams["pdf.fonttype"] = 42  # for pdfs

from iss_preprocess.io import get_processed_path

In [ ]:
processed_path = get_processed_path("becalia_rabies_barseq/BRAC8498.3e/analysis")

In [ ]:
error_correction_ds_name = "BRAC8498.3e_error_corrected_barcodes_26"
df_path = processed_path / f"{error_correction_ds_name}_cell_barcode_df.pkl"
# Run only once to add gene info
if False:
    old_df = pd.read_pickle(processed_path / "cell_barcode_df.pkl")
    from brisc.manuscript_analysis.cell_barcode_assignment import assign_cell_barcodes

    new_df = assign_cell_barcodes(
        error_correction_ds_name=error_correction_ds_name,
    )

    col2copy = ["raw_gene_counts", "best_score", "knn_agree_conf", "best_cluster"]
    new_df = pd.merge(
        new_df, old_df[col2copy], how="left", left_index=True, right_index=True
    )
    new_df.to_pickle(df_path)
    coords = new_df[[f"ara_{o}" for o in "xyz"]]
    print(coords.isna().values.any(1).sum())

In [ ]:
print("Loading data")
cells_df = load.load_cell_barcode_data(
    processed_path,
    areas_to_empty=["fiber tracts", "outside"],
    valid_areas=["Isocortex", "TH"],
    distance_threshold=150,
    error_correction_ds_name=error_correction_ds_name,
)

In [ ]:
print("Projection on flatmap")
flat_coors = compute_flatmap_coors(cells_df, distance_cutoff=150)
cells_df["flatmap_x"] = flat_coors[:, 0]
cells_df["flatmap_y"] = flat_coors[:, 1]
cells_df["flatmap_z"] = flat_coors[:, 2]

normalised_coors = compute_flatmap_coors(cells_df, distance_cutoff=150, thickness_type='normalized_layers')
# keep x/y too even if they're the same as non normalised to make it easier for later
# functions

cells_df["flatmap_x_normalised"] = np.array(normalised_coors[:, 0])
cells_df["flatmap_y_normalised"] = np.array(normalised_coors[:, 1])
cells_df["flatmap_z_normalised"] = np.array(normalised_coors[:, 2])


In [ ]:
cells_df.columns

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,1,1, aspect='equal')
moved = cells_df.query('was_in_wm == True')
non_moved = cells_df.query('was_in_wm == False')
plt.scatter(non_moved.ara_z, non_moved.ara_y)
plt.scatter(moved.ara_z, moved.ara_y)
plt.xlim(4,22)
plt.ylim(5, 0)

plt.scatter(non_moved.flatmap_x/100+10, non_moved.flatmap_z_normalised/100)
plt.scatter(moved.flatmap_x/100+10, moved.flatmap_z_normalised/100)
for _, m in moved.iterrows():
    plt.plot([m.ara_z, m.flatmap_x/100+10],[m.ara_y, m.flatmap_z_normalised/100],color='k', alpha=0.2)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,1,1, aspect='equal')
moved = cells_df.query('was_in_wm == True')
non_moved = cells_df.query('was_in_wm == False')
for _, m in moved.iterrows():
    plt.plot([m.ara_z, m.flatmap_x/100+10],[m.ara_y, m.flatmap_z/100],color='k', alpha=0.2)
plt.scatter(non_moved.ara_z, non_moved.ara_y)
plt.scatter(moved.ara_z, moved.ara_y)
plt.xlim(4,22)

plt.scatter(non_moved.flatmap_x/100+10, non_moved.flatmap_z/100)
plt.scatter(moved.flatmap_x/100+10, moved.flatmap_z/100)


In [ ]:
projection = "flatmap_dorsal"
thickness_type = 'normalized_layers'
from brisc.exploratory_analysis import plot_summary_for_all_bc as psa
ori_coors = moved[[f"ara_{c}" for c in 'xyz']].values 
flat_coors = np.zeros_like(ori_coors)
bad = np.any(ori_coors <= 0, axis=1)
# also remove NaN
bad = np.logical_or(bad, np.any(np.isnan(ori_coors), axis=1))
if np.any(bad):
    print(f"Found {np.sum(bad)} bad coordinates")
    flat_coors[bad, :] = np.nan
ccf_coord_proj = psa.get_projector(projection)
if projection == "flatmap_dorsal":
    view_space = "flatmap_dorsal"
elif projection == "top":
    view_space = False
else:
    print(f"Warning unknown projections: {projection}.")
    view_space = False
print(np.sum(bad), len(ori_coors))
flat_coors.shape


In [ ]:
flat_coors[~bad, :] = ccf_coord_proj.project_coordinates(
        ori_coors[~bad, :] * 1000,
        drop_voxels_outside_view_streamlines=False,
        view_space_for_other_hemisphere=view_space,
        hemisphere="right",
        thickness_type=thickness_type,
    )

In [ ]:
to_move = np.isnan(flat_coors[:, 0])
print(to_move.sum())

In [ ]:
plt.subplot(1,1,1, aspect='equal')
valid_voxels = ccf_coord_proj.closest_surface_voxels[:, 0]
valid_voxels = np.vstack(
    np.unravel_index(valid_voxels, np.array(ccf_coord_proj.volume_shape))
).T
# Find voxels of points to move
ori_voxel = (ori_coors[to_move, :] * 1000 / ccf_coord_proj.resolution).astype(int)
plt.scatter(ori_voxel[:,2], ori_voxel[:,1])
z_size = ccf_coord_proj.volume_shape[2]
z_midline = z_size / 2
# reflect to left hemisphere
to_reflect = ori_voxel[:, 2] > z_midline
ori_voxel[to_reflect, 2] = z_size - ori_voxel[to_reflect, 2]
plt.scatter(ori_voxel[:,2], ori_voxel[:,1])
plt.axvline(z_midline)

In [ ]:
from tqdm import tqdm
distance_cutoff = 150
min_v, max_v = ori_voxel.min(axis=0), ori_voxel.max(axis=0)
dst_cutoff = distance_cutoff / ccf_coord_proj.resolution[0]
print(dst_cutoff)
print(min_v)
print(max_v)
lower_bounds = min_v - dst_cutoff
upper_bounds = max_v + dst_cutoff
is_within_bounds = np.all(
    (valid_voxels >= lower_bounds) & (valid_voxels <= upper_bounds), axis=1
)
valid_voxels = valid_voxels[is_within_bounds, :]
distances, closest_voxels = np.zeros(ori_voxel.shape[0]), np.zeros_like(ori_voxel)
for i, voxel in tqdm(
    enumerate(ori_voxel),
    total=ori_voxel.shape[0],
    desc="Find closest voxel that can be projected to flatmap",
):
    # keep valid_voxels in a dst_cutoff cube around voxel
    upper = voxel + dst_cutoff
    lower = voxel - dst_cutoff
    is_within = np.all((valid_voxels >= lower) & (valid_voxels <= upper), axis=1)
    vvox = valid_voxels[is_within, :]
    dst = np.linalg.norm(vvox - voxel, axis=-1)
    if not len(dst):
        distances[i] = np.inf
        continue
    distances[i] = dst.min()
    closest_voxels[i] = vvox[dst.argmin(), :]
close_enough = distances < dst_cutoff
print(f"Found {np.sum(close_enough)}/{ori_voxel.shape[0]} close enough")
# put them back in the right hemisphere if needed
closest_reflected = closest_voxels.copy()
closest_reflected[to_reflect, 2] = z_size - closest_reflected[to_reflect, 2]


In [ ]:
plt.subplot(1,1,1, aspect='equal')
for o, c in zip(ori_voxel[close_enough], closest_voxels[close_enough]):
    plt.plot([o[2], c[2]], [o[1], c[1]], color='k')
plt.scatter(ori_voxel[close_enough,2], ori_voxel[close_enough,1])
plt.scatter(closest_voxels[close_enough,2], closest_voxels[close_enough,1])


In [ ]:
reprojected = np.empty(closest_voxels.shape) + np.nan
reprojected[close_enough, :] = ccf_coord_proj.project_coordinates(
    (closest_reflected[close_enough, :] * ccf_coord_proj.resolution[0]).astype(float),
    drop_voxels_outside_view_streamlines=False,
    view_space_for_other_hemisphere=view_space,
    hemisphere="both",
)


In [ ]:
ccf_coord_proj.resolution[0]

In [ ]:
ctx_cells = cells_df[
    (~cells_df["cortical_area"].isna())
    & ~(cells_df.cortical_area.isin(["TH", "hippocampal"]))
]
print(
    f"{ctx_cells.flatmap_x.isna().sum()}/{ctx_cells.shape[0]} cortical cells have no flatmap coordinates"
)
print("Matching barcodes")
conn_mat.match_barcodes(cells_df)
dist.add_connection_distances(cells_df, cols=["flatmap_x", "flatmap_y"])

In [ ]:
non_excitatory_cell_types = ["Vip", "Sst", "Pvalb", "Lamp5", "VLMC"]
non_excitatory = cells_df["best_cluster"].isin(non_excitatory_cell_types)

cells_df = cells_df[~non_excitatory]
layers = ["L2/3", "L4", "L5", "L6a", "L6b"]
# local connections
cells_df = cells_df[cells_df["cortical_layer"].notnull()]
# change any cells with cells_df["cortical_layer"] == "L1" to "L2,3"
cells_df.loc[cells_df["cortical_layer"] == "L1", "cortical_layer"] = "L2/3"
cells_df = cells_df[cells_df["cortical_layer"].apply(lambda layer: layer in layers)]
cells_df = cells_df[cells_df["distances"].apply(lambda dist: np.max(dist)) < 100]

grouping = "cortical_layer"  # "area_acronym_ancestor_rank1"
# Shuffle the barcodes assigned to each cell in the connectivity matrix
(
    shuffled_cell_barcode_dfs,
    shuffled_matrices,
    mean_input_fraction_dfs,
    starter_input_fractions,
    _,
) = conn_mat.shuffle_and_compute_connectivity(
    cells_df,
    n_permutations=1000,
    shuffle_starters=False,
    shuffle_presyn=True,
    starter_grouping=grouping,
    presyn_grouping=grouping,
    output_fraction=False,
)

(
    shuffled_cell_barcode_dfs,
    shuffled_matrices,
    output_fraction_dfs,
    _,
    _,
) = conn_mat.shuffle_and_compute_connectivity(
    cells_df,
    n_permutations=1000,
    shuffle_starters=False,
    shuffle_presyn=True,
    starter_grouping=grouping,
    presyn_grouping=grouping,
    output_fraction=True,
)

connectivity_matrix, mean_input_fraction, fractions_df, _ = (
    conn_mat.compute_connectivity_matrix(
        cells_df,
        starter_grouping=grouping,
        presyn_grouping=grouping,
        output_fraction=False,
    )
)

connectivity_matrix, output_fraction, _, _ = conn_mat.compute_connectivity_matrix(
    cells_df,
    starter_grouping=grouping,
    presyn_grouping=grouping,
    output_fraction=True,
)

In [ ]:
input_fraction_log_ratio, input_fraction_pval = conn_mat.compare_to_shuffle(
    *conn_mat.filter_matrices(
        mean_input_fraction,
        np.array(mean_input_fraction_dfs),
        row_order=layers,
        col_order=layers,
    )
)

output_fraction_log_ratio, output_fraction_pval = conn_mat.compare_to_shuffle(
    *conn_mat.filter_matrices(
        output_fraction,
        np.array(output_fraction_dfs),
        row_order=layers,
        col_order=layers,
    )
)

In [ ]:
layers = ["L2/3", "L4", "L5", "L6a", "L6b"]

counts_df, mean_input_frac_df, fractions_df, _ = conn_mat.compute_connectivity_matrix(
    cells_df,
    starter_grouping="cortical_layer",  # "area_acronym_ancestor_rank1",
    presyn_grouping="cortical_layer",  # "area_acronym_ancestor_rank1",
)

from tqdm import tqdm

# Create bootstrap samples
nboot = 1000
bootstrap_samples = []

for i in tqdm(range(nboot)):
    rows = []
    for layer, group in fractions_df.groupby("cortical_layer", observed=True):
        this_layer = group.sample(n=len(group), replace=True)[layers].mean()
        this_layer.name = layer
        rows.append(this_layer)  # Resample with replacement
    bootstrap_samples.append(pd.concat(rows, axis=1))
bootstrap_samples = np.array(bootstrap_samples)
lower_df = pd.DataFrame(
    data=np.quantile(bootstrap_samples, 0.025, axis=0), index=layers, columns=layers
)
upper_df = pd.DataFrame(
    data=np.quantile(bootstrap_samples, 0.975, axis=0), index=layers, columns=layers
)

In [ ]:
areas = {
    "L2/3": "2/3",
    "L4": "4",
    "L5": "5",
    "L6a": "6a",
    "L6b": "6b",
}
presynaptic_counts = conn_mat.reorganise_matrix(counts_df).sum(axis=1)
starter_counts = fractions_df.value_counts("cortical_layer").rename(index=areas)

In [ ]:
from brisc.manuscript_analysis import distance_between_cells as dist_cells

relative_presyn_coords_flatmap, distancess_flatmap, starters_df = (
    dist_cells.determine_presynaptic_distances(cells_df, col_prefix="flatmap_", col_suffix='_normalised')
)
alllayers = ["L1"] + layers + ["WM"]
starters_df["layer_index"] = starters_df.cortical_layer.map(
    lambda x: alllayers.index(x) if x in alllayers else np.nan
)

In [ ]:
# Plot Fig.1
save_fig = False

fontsize_dict = {"title": 7, "label": 7, "tick": 6, "legend": 6}
pad_dict = {"label": 1, "tick": 1, "legend": 5}
line_width = 0.9
line_alpha = 1

cm = 1 / 2.54
fig = plt.figure(figsize=(17.4 * cm, 17.4 * cm), dpi=200)

save_path = Path("Z:/home/shared/presentations/becalick_2025")
# save_path = Path("/nemo/lab/znamenskiyp/home/shared/presentations/becalick_2025")


figname = "matrices"


if True:
    frame = fig.add_axes([0, 0, 1, 1])
    frame.set_xticks([])
    frame.set_yticks([])

# Presynaptic scatters
w = 0.17
scl = 10  # scale to put distance in um
avg_layer_tops = get_avg_layer_depth()
layer_borders = np.hstack([0, np.sort(np.hstack(list(avg_layer_tops.values()))), 1200])
midlayer = np.diff(layer_borders) / 2 + layer_borders[:-1]
ax_layers = []

for il, layer in enumerate(layers):
    axl = fig.add_axes([0.05 + (w + 0.01) * il, 0.6, w, 0.2], aspect="equal")
    for d in layer_borders[:-1]:
        axl.axhline(d, ls="--", lw=0.5, color="lightgray")
    this_layer = starters_df[starters_df["cortical_layer"] == layer]

    rel_ap = np.hstack(this_layer["presynaptic_coors_relative"].values)[0][:, 0] * scl
    rel_ml = np.hstack(this_layer["presynaptic_coors_relative"].values)[0][:, 1] * scl
    abs_depth = np.hstack(this_layer["presynaptic_coors"].values)[0][:, 2] * scl
    plt.scatter(
        rel_ml, abs_depth, marker=".", color="darkred", alpha=0.5, ec="w", lw=0.1, s=10
    )
    plt.scatter(
        np.zeros(len(this_layer)),
        this_layer["flatmap_z_normalised"] * scl,
        marker=".",
        color="k",
        s=20,
        alpha=0.3,
        ec="None",
    )
    axl.set_xlim(-800, 800)
    axl.set_ylim(1350, -50)
    if il == 0:
        axl.set_title(
            f"Starter layer:     {layer[1:]}", fontsize=fontsize_dict["title"]
        )
        rec = plt.Rectangle((-680, 1200), 200, 30, color="k")
        axl.add_artist(rec)
    else:
        axl.set_title(layer[1:], fontsize=fontsize_dict["title"])
    axl.set_axis_off()
for il, layer in enumerate(["L1"] + layers + [" WM"]):
    axl.text(
        750,
        midlayer[il],
        layer[1:],
        fontsize=fontsize_dict["legend"],
        horizontalalignment="center",
        verticalalignment="center",
    )


# Raw counts
ax_counts = fig.add_axes([0.07, 0.35, 0.2, 0.2])
conn_mat.plot_area_by_area_connectivity(
    conn_mat.reorganise_matrix(counts_df, areas=areas),
    starter_counts,
    presynaptic_counts,
    ax_counts,
    input_fraction=False,
    odds_ratio=False,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    line_width=line_width,
)

# Input fraction
ax_input_fraction = fig.add_axes([0.28, 0.35, 0.2, 0.2])
ax_input_fraction_cb = fig.add_axes([0.49, 0.35, 0.01, 0.05])
conn_mat.plot_area_by_area_connectivity(
    conn_mat.reorganise_matrix(mean_input_fraction, areas=areas),
    starter_counts,
    presynaptic_counts,
    ax_input_fraction,
    input_fraction=True,
    odds_ratio=False,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    line_width=line_width,
    show_counts=False,
    cbax=ax_input_fraction_cb,
    cbar_label="Input\nfraction",
)
ax_input_fraction.set_ylabel("")
ax_input_fraction.set_yticks([])

# Confidence interval of input fraction
ax_input_fraction_bars = fig.add_axes([0.61, 0.37, 0.25, 0.18])
boot.plot_confidence_intervals(
    conn_mat.reorganise_matrix(mean_input_frac_df, areas=areas),
    conn_mat.reorganise_matrix(lower_df, areas=areas),
    conn_mat.reorganise_matrix(upper_df, areas=areas),
    ax_input_fraction_bars,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    line_width=line_width,
    orientation="horizontal",
)

# Schematics
ax_schematics = fig.add_axes([0.86, 0.39, 0.15, 0.18])
cax_schematics = fig.add_axes([0.89, 0.34, 0.01, 0.05])
positions = {l: p for l, p in zip(layers, [(2, 6), (0, 5), (2, 4), (0, 3), (2, 2)])}
fig, ax, cbar = conn_mat.connectivity_diagram_mpl(
    mean_input_fraction,
    lower_df,
    upper_df,
    connection_names=layers,
    positions=positions,
    display_names=[l[1:] for l in layers],
    node_style=dict(facecolor="Lightgray", radius=0.5, fontsize=fontsize_dict["title"]),
    min_fraction_cutoff=0.2,
    ci_to_alpha=False,
    ci_cmap="plasma",
    edge_width_scale=10,
    arrow_head_scale=20,
    arrow_style=dict(connectionstyle="Arc3, rad=-0.2", ec="none"),
    ax=ax_schematics,
    cax=cax_schematics,
)
# cbar.set_ticks([0, 0.5, 1])


# Bubble plot input fraction
ax_bubble_plot_input_frac = fig.add_axes([0.07, 0.05, 0.2, 0.2])
ax_bubble_plot_input_frac_cb = fig.add_axes([0.29, 0.2, 0.01, 0.05])
conn_mat.bubble_plot(
    conn_mat.reorganise_matrix(input_fraction_log_ratio, areas=areas),
    conn_mat.reorganise_matrix(input_fraction_pval, areas=areas),
    alpha=0.05,
    size_scale=250,
    ax=ax_bubble_plot_input_frac,
    cbax=ax_bubble_plot_input_frac_cb,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
)

# Output fraction
ax_output_fraction = fig.add_axes([0.45, 0.05, 0.2, 0.2])
ax_output_fraction_cb = fig.add_axes([0.66, 0.05, 0.01, 0.05])
# output_fraction = conn_mat.reorganise_matrix(output_fraction)
conn_mat.plot_area_by_area_connectivity(
    conn_mat.reorganise_matrix(output_fraction, areas=areas),
    starter_counts,
    presynaptic_counts,
    ax_output_fraction,
    cbax=ax_output_fraction_cb,
    cbar_label="Output\nfraction",
    input_fraction=True,
    odds_ratio=False,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    line_width=line_width,
    show_counts=False,
)

# Bubble plot input fraction
ax_bubble_plot_output_frac = fig.add_axes([0.78, 0.05, 0.2, 0.2])
conn_mat.bubble_plot(
    conn_mat.reorganise_matrix(output_fraction_log_ratio, areas=areas),
    conn_mat.reorganise_matrix(output_fraction_pval, areas=areas),
    alpha=0.05,
    size_scale=250,
    ax=ax_bubble_plot_output_frac,
    show_legend=False,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
)

if save_fig:
    fig.savefig(save_path / f"conn_mat.pdf", format="pdf")
    fig.savefig(save_path / f"conn_mat.png", format="png")

In [ ]:
layers

In [ ]:
alllayers = ["L1"] + layers + ["WM"]
starters_df["layer_index"] = starters_df.cortical_layer.map(
    lambda x: layers.index(x) if x in alllayers else np.nan
)

In [ ]:
import brainglobe_atlasapi as bga

atlas = bga.BrainGlobeAtlas(atlas_name="allen_mouse_10um")

In [ ]:
st = cells_df.query("is_starter==True")
st_coords = st[["ara_x", "ara_y", "ara_z"]].values * 1000

In [ ]:
areas = [
    atlas.structure_from_coords(c, as_acronym=True, microns=True) for c in st_coords
]

In [ ]:
st["area_rematch"] = areas
st.loc[(st["area_acronym"] != st["area_rematch"]), ["area_acronym", "area_rematch"]]

In [ ]:
from brisc.manuscript_analysis import distance_between_cells as dist_cells

relative_presyn_coords_flatmap, distancess_flatmap, starters_df = (
    dist_cells.determine_presynaptic_distances(cells_df, col_prefix="flatmap_")
)

plt.figure(figsize=(10, 5))
nboot = 1000
dist = np.zeros(len(layers))
dist_boot = np.zeros((len(layers), nboot))


def get_median_dist(starters):
    relative_ap = np.hstack(starters["presynaptic_coors_relative"].values)[0][:, 0]
    relative_ml = np.hstack(starters["presynaptic_coors_relative"].values)[0][:, 1]
    distances = ((relative_ml**2 + relative_ap**2) ** 0.5).astype(float)
    return np.nanmedian(distances)


for i, layer in enumerate(layers):

    plt.subplot(2, 6, i + 1)
    this_layer = starters_df[starters_df["cortical_layer"] == layer]
    rel_ap = np.hstack(this_layer["presynaptic_coors_relative"].values)[0][:, 0]
    rel_ml = np.hstack(this_layer["presynaptic_coors_relative"].values)[0][:, 1]

    abs_depth = np.hstack(this_layer["presynaptic_coors"].values)[0][:, 2]
    for iboot in range(nboot):
        dist_boot[i, iboot] = get_median_dist(
            this_layer.sample(replace=True, n=len(this_layer), axis=0)
        )
    plt.plot(rel_ml, abs_depth, ",k", alpha=0.5)
    # plt.plot(relative_ap, relative_ml, ',k', alpha=0.5)
    plt.plot(np.zeros(len(this_layer)), this_layer["flatmap_z"], ".", color="r")

    plt.gca().invert_yaxis()
    plt.axis("equal")
    plt.axis("off")
    plt.xlim([-100, 100])
    plt.ylim([100, 0])

    distances = ((rel_ml**2 + rel_ap**2) ** 0.5).astype(float)
    dist[i] = np.nanmedian(distances)
    print(dist[i])